In [1]:
import numpy as np
import sys
import os

np.set_printoptions(threshold=np.inf, linewidth=np.inf)

# Absolute path to the folder that contains "quant_rotor"
sys.path.append("/Users/gilfrim/Desktop/Projects/Computational_Analysis_of_Many_Rotor_Systems")

# Set print output parameters
np.set_printoptions(suppress = True, linewidth = np.inf, threshold = np.inf, precision = 12)


#working with kietic and potential operators

# Dense (Slow)
from quant_rotor.models.dense.support_ham import write_matrix_elements, basis_m_to_p_matrix_conversion

# Sparse (Fast)
from quant_rotor.models.sparse.support_ham import build_V_in_p

# Density Matrices

from quant_rotor.models.dense.density_matrix import density_matrix_1, density_matrix_2, dencity_energy

# ED hamiltonian implementation.

# Dense (Slow)
from quant_rotor.core.dense.hamiltonian import hamiltonian_dense
from quant_rotor.core.dense.hamiltonian_big import hamiltonian_general_dense

# Sparse (Fast)
from quant_rotor.core.sparse.hamiltonian import hamiltonian_sparse
from quant_rotor.core.sparse.hamiltonian_big import hamiltonian_general_sparse


# Iterative CCC calculation

# Dense (Slow)
from quant_rotor.core.dense.t_amplitudes_guess import amplitute_energy, t_1_amplitude_guess_ground_state, t_2_amplitude_guess_ground_state, intermediate_normalisation
from quant_rotor.core.dense.t_amplitudes_periodic import t_periodic as t_periodic_dense
from quant_rotor.core.dense.t_amplitudes_non_periodic import t_non_periodic as t_non_periodic_dense

# Sparse (Fast)
from quant_rotor.core.dense.t_amplitudes_periodic_fast import t_periodic as t_periodic_sparse # for bigger systems (much harder to modify)


#Time dependant methods

# Dense (Slow)
from quant_rotor.core.dense.de_solve_one_thermal import integration_scheme

# Sparse (Fast)
from quant_rotor.core.dense.de_solve_one_thermal_dense import integration_scheme as integration_scheme_fast # for bigger systems (much harder to modify)

**!!!Before you start make sure to change the path in the first cell line 8 to the one from your computer!!!**

# Producing hamiltonian operators.

## Producing Kinetic and Potential operators.

In [2]:
state = 3
g = 0.1
tau = 0

Example of the call to produce kinetic and potential matricies.

In [3]:
# Recomended to produce small matrix which can be printed and see how it changes. For example 3 state system

Kinetic, Potential = write_matrix_elements((state-1)//2, tau) # takes input of the number of unique states see doc-string and annotation for more information

K, V = Kinetic, Potential

V_tensor = V.reshape(state, state, state, state)  # Adjust if needed

h_full = basis_m_to_p_matrix_conversion(K, state)
v_full = basis_m_to_p_matrix_conversion(V_tensor, state)

In [4]:
h_full

array([[0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j]])

In [5]:
v_full.reshape(state**2, state**2)

array([[ 0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.75+0.j, -0.25+0.j,  0.  +0.j, -0.25+0.j,  0.75+0.j],
       [ 0.  +0.j,  0.  +0.j,  0.  +0.j, -0.25+0.j,  0.  +0.j,  0.  +0.j,  0.75+0.j,  0.  +0.j,  0.  +0.j],
       [ 0.  +0.j,  0.  +0.j,  0.  +0.j,  0.75+0.j,  0.  +0.j,  0.  +0.j, -0.25+0.j,  0.  +0.j,  0.  +0.j],
       [ 0.  +0.j, -0.25+0.j,  0.75+0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j],
       [ 0.75+0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j],
       [-0.25+0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j],
       [ 0.  +0.j,  0.75+0.j, -0.25+0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j],
       [-0.25+0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j],
       [ 0.75+0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j]])

In [6]:
Kinetic_sparse, Potential_sparse = build_V_in_p(state, tau) # the fast version give a result in sparse which needs to ve converted to dense matricies

# Example of how to convert to dense.

Kinetic_dense = Kinetic_sparse.toarray()
Potential_dense = Potential_sparse.toarray()

/Users/gilfrim/Desktop/Projects/Computational_Analysis_of_Many_Rotor_Systems/.venv/lib/python3.13/site-packages/scipy/sparse/_sputils.py:147: ComplexWarning: Casting complex values to real discards the imaginary part
  data = np.array(obj, dtype=dtype, copy=copy)


In [7]:
Kinetic_dense

array([[0., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [8]:
Potential_dense

array([[ 0.  ,  0.  ,  0.  ,  0.  ,  0.75, -0.25,  0.  , -0.25,  0.75],
       [ 0.  ,  0.  ,  0.  , -0.25,  0.  ,  0.  ,  0.75,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.75,  0.  ,  0.  , -0.25,  0.  ,  0.  ],
       [ 0.  , -0.25,  0.75,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.75,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [-0.25,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.75, -0.25,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [-0.25,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.75,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ]])

Check the output.

In [9]:
print(np.array_equal(h_full, Kinetic_dense))
print(np.allclose(v_full.reshape(state**2, state**2), Potential_dense), 1e-30)

True
True 1e-30


Produce several matricies for tests.

In [10]:
save_dir = "example_data/kinetic_potential_matricies/ED"

for state in range(3, 15, 2):

    Kinetic_sparse, Potential_sparse = build_V_in_p(state, tau) # the fast version give a result in sparse which needs to ve converted to dense matricies

    # Example of how to convert to dense.

    Kinetic_dense = Kinetic_sparse.toarray()
    Potential_dense = Potential_sparse.toarray()

    save_path = os.path.join(save_dir, f"kin_pot_ED_state_{state}.npz")
    np.savez(save_path, K=Kinetic_dense, V=Potential_dense)

In [11]:
save_dir = "example_data/kinetic_potential_matricies/ED"

for state in range(3, 15, 2):

    Kinetic_sparse, Potential_sparse = build_V_in_p(state, tau) # the fast version give a result in sparse which needs to ve converted to dense matricies

    # Example of how to convert to dense.

    Kinetic_dense = Kinetic_sparse.toarray()
    Potential_dense = Potential_sparse.toarray()

    save_path = os.path.join(save_dir, f"kin_pot_ED_state_{state}.npz")

    data = np.load(save_path)

    print(np.array_equal(Kinetic_dense, data["K"]))
    print(np.array_equal(Potential_dense, data["V"]))

True
True
True
True
True
True
True
True
True
True
True
True


## Producing Exact Diagonalization hamiltonians.

In [6]:
site = 3
state = 5
g = 1

Exact Diagonalization hamiltonian dense (slow).

In [13]:
H_ED_d, K_ED_d, V_ED_d = hamiltonian_dense(state, site, g, np.pi / 2)

eig_val_ED_d, eig_vec_ED_d = np.linalg.eigh(H_ED_d) # produces the solutions to the hamiltonian with energys and associated wavefunctions.

In [14]:
V_ED_d.real

array([[0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,

Exact Diagonalization hamiltonian sparse (fast).

In [12]:
H_ED_s, K_ED_s, V_ED_s = hamiltonian_sparse(state, site, g)

H_ED_s = H_ED_s.toarray()
K_ED_s = K_ED_s.toarray()
V_ED_s = (V_ED_s.toarray())

eig_val_ED_s, eig_vec_ED_s = np.linalg.eigh(H_ED_s) # produces the solutions to the hamiltonian with energys and associated wavefunctions.

In [10]:
V_ED_s

array([[ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.75, -0.25,  0.  ,  0.  ,  0.  , -0.25,  0.75,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  , -0.25,  0.  ,  0.  ,  0.75,  0.  ,  0.75,  0.  ,  0.  , -0.25,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.75,  0.  ,  0.  ,  0.  , -0.25, -0.25,  0.  ,  0.  ,  0.  ,  0.75,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  , -0.25,  0.  ,  0.  ,  0.  ,  0.  ,  0.75,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.75,  0.  ,  0.  ,  0.  ,  0.  , -0.25,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  , -0.25,  0.75,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0

In [ ]:
psi_twist

Check agains each other. (Should match)

In [11]:
print("H_ED:", np.allclose(H_ED_d, H_ED_s))
print("K_ED:", np.allclose(K_ED_d, K_ED_s))
print("V_ED:", np.allclose(V_ED_d, V_ED_s))
print("eigen values:", np.allclose(eig_val_ED_d, eig_val_ED_s))
print("eigen vectors:", np.allclose(eig_vec_ED_d, eig_vec_ED_s))

H_ED: False
K_ED: True
V_ED: False
eigen values: True
eigen vectors: False


## Density matrices

In [19]:
site = 3
state = 11
g = 1

First for the density we extract a ground state vector.

In [20]:
HKV = hamiltonian_dense(state, site, g)
H = HKV[0]

eig_val, eig_vec = np.linalg.eigh(H)

index = np.argmin(eig_val)
ground_state_vec = eig_vec[:, index]

KeyboardInterrupt: 

Then we calculate densities for one site and two site interactions.

In [ ]:
D_1 = density_matrix_1(state, site, ground_state_vec, 0)
D_2 = density_matrix_2(state, site, ground_state_vec, 0, 1)

eig_val_D_1, eig_vec_D_1 = np.linalg.eigh(D_1)
eig_val_D_2, eig_vec_D_2 = np.linalg.eigh(D_2.reshape(state**2, state**2))

Two easy checks we can do to make sure the density matrices are correct are to sup up the eigenstates; they should roughly add up to one.

In [ ]:
print("Single density:", np.sum(eig_val_D_1))
print("Double density:", np.sum(eig_val_D_2))

Single density: 1.0000000000000002
Double density: 1.0000000000000004


The second check we can do is to we expect the two large identical eigenvalues for single and double interaction for the large g.

This makes physical sense since with large interaction strength (determined by g) the system roughly acts as polarized magnets aligning in one or the other
direction of the polarity. So our quantum system is in the superposition of those two alignments.

In [ ]:
-np.sort(-eig_val_D_1)
-np.sort(-eig_val_D_2)

array([ 0.60219817545 ,  0.388316946381,  0.006905393475,  0.001484486549,  0.001067865619,  0.000020019676,  0.000007011748,  0.000000056077,  0.000000044874,  0.000000000091,  0.00000000006 ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.            ,  0.    

Additional check that can be done as an exercise is to check whether the eigenvectors of D_1 and D_2 (separately) are orthogonal.

## Producing Natural Orbitas hamiltonians

Natural Orbitals hamiltonian dense (slow).

In [ ]:
H_NO_d, K_NO_d, V_NO_d = hamiltonian_general_dense(state, site, g)

eig_val_NO_d, eig_vec_NO_d = np.linalg.eigh(H_NO_d) # produces the solutions to the hamiltonian with energys and associated wavefunctions.

A way to check Natural Orbitals approximation with exact Diagonalization is to set the states = 11 and sites = 3.

The resulting Hamiltonians from ED and NO should have the same energy states (eigenvalues/eigenstates).

In [ ]:
np.allclose(eig_val_NO_d, eig_val_ED_s, atol=1e-19)

False

Natural Orbitals hamiltonian sparse (fast).

In [ ]:
H_NO_s, K_NO_s, V_NO_s = hamiltonian_general_sparse(state, site, g)

eig_val_NO_s, eig_vec_NO_s = np.linalg.eigh(H_NO_s.toarray()) # produces the solutions to the hamiltonian with energys and associated wavefunctions.

/Users/gilfrim/Desktop/Projects/Computational_Analysis_of_Many_Rotor_Systems/.venv/lib/python3.13/site-packages/scipy/sparse/_sputils.py:147: ComplexWarning: Casting complex values to real discards the imaginary part
  data = np.array(obj, dtype=dtype, copy=copy)


Check the Natural Orbitals methods against each other.

In [ ]:
np.allclose(eig_val_NO_d, eig_val_NO_s, atol=1e-19)

True

# Extracting t amplitudes.

First produce the hamiltonian, diagonalize and extract ground state.

In [ ]:
state = 11
site = 3
g = 0.1

In [ ]:
H, K, V = hamiltonian_general_dense(state, site, g)
eig_val_ED, eig_vec_ED = np.linalg.eigh(H)

Produce a ground state in the intermediate diagonalization.

In [ ]:
ground_norm = intermediate_normalisation(eig_val_ED, eig_vec_ED)

Produce the t_1 and t_2 amplitudes.

In [ ]:
t_a_i_tensor = t_1_amplitude_guess_ground_state(state, site, g, eig_vec, eig_val) # shape (site, state - 1, 1)
t_ab_ij_tensor = t_2_amplitude_guess_ground_state(state, site, g, eig_vec, eig_val) # shape (site, site, state - 1, state - 1, 1, 1)

We can reduce the dimension of tensors by reshaping.

# Twisted Hamiltonian.

In [ ]:
state = 5
site = 3
g = 0.1

In [ ]:
H, K, V = hamiltonian_dense(state, site, g, np.pi)

In [ ]:
K

array([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 4.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 4.+0.j]])

In [ ]:
V

array([[ 0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.075+0.j, -0.025+0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j, -0.025+0.j,  0.075+0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j],
       [ 0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j, -0.025+0.j,  0.   +0.j,  0.   +0.j,  0.075+0.j,  0.   +0.j,  0.075+0.j,  0.   +0.j,  0.   +0.j, -0.025+0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j],
       [ 0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.075+0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j, -0.025+0.j, -0.025+0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.075+0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j],
       [ 0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.  